Subir el audio.zip y synthesized_audios.zip

# Install packages

In [ ]:
!pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 7.1 MB/s eta 0:00:00


In [ ]:
!pip install -U openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 7.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.6 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801356 sha256=cf779f784443e9d7cf1c97e9d6e20cd67a409b4c70f685b78334c4293fd28430
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


# Import real audios and synthesized audios

In [ ]:
import zipfile
import os
import shutil
import glob
import pandas as pd
import numpy as np

def unzip_files(zip_file_path):

  destination_folder = zip_file_path.split('.zip')[0]

  # Create the destination folder if it doesn't exist
  if not os.path.exists(destination_folder):
      os.makedirs(destination_folder)

  # Open the zip file
  with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
      # Extract all contents to the destination folder
      zip_ref.extractall(destination_folder)

  print(f"Contents of '{zip_file_path}' extracted to '{destination_folder}'.")

In [ ]:
unzip_files('audio.zip')
os.remove('audio.zip')
shutil.rmtree('sample_data')

Contents of 'audio.zip' extracted to 'audio'.


In [ ]:
unzip_files('synthesized_audios.zip')
os.remove('synthesized_audios.zip')

Contents of 'synthesized_audios.zip' extracted to 'synthesized_audios'.


In [ ]:
import os
import shutil

# Define the paths
outer_folder = 'synthesized_audios'
inner_folder = os.path.join(outer_folder, outer_folder)

# Move the contents of the inner folder to the outer folder
for item in os.listdir(inner_folder):
    source_path = os.path.join(inner_folder, item)
    destination_path = os.path.join(outer_folder, item)
    shutil.move(source_path, destination_path)

# Remove the inner folder
shutil.rmtree(inner_folder)

print("Folders rearranged successfully.")


Folders rearranged successfully.


# Obtain transcriptions

In [ ]:
import whisper

whisper = whisper.load_model("base")

def speech_to_text(metadata_csv_path):

  metadata_df = pd.read_csv(metadata_csv_path)

  if 'synthesized' in metadata_csv_path:
    dataset = metadata_csv_path.split('/')[-1].split('_')[0]
    model = metadata_csv_path.split('/')[1]
    audios_path = os.path.join('synthesized_audios', model, dataset)
    audios_colname = 'Synthesized audio ID'
  else:
    audios_path = metadata_csv_path.split('_metadata.csv')[0]
    audios_colname = 'Audio ID'

  csv_name = metadata_csv_path.split('/')[-1]

  for index, row in metadata_df.iterrows():

    audio_path = os.path.join(audios_path, row[audios_colname] +'.wav')

    asr_text = whisper.transcribe(audio_path)['text']

    metadata_df.loc[index, 'Transcription'] = asr_text

  metadata_df.to_csv(os.path.join('metadata_complete_aux', csv_name), index=False)

In [ ]:
# shutil.rmtree('metadata_complete_aux')

In [ ]:
metadata_csv_paths = glob.glob('**/*.csv', recursive=True)
os.makedirs('metadata_complete_aux')

metadata_csv_paths = [csv for csv in os.listdir('synthesized_audios/TorToise') if csv.count('_') == 2]
metadata_csv_paths = [os.path.join('synthesized_audios/TorToise', path) for path in metadata_csv_paths]

for metadata_csv_path in metadata_csv_paths:
  speech_to_text(metadata_csv_path)

/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
/usr/local/lib/

In [ ]:
pd.read_csv('/content/metadata_complete_aux/Spanish_YourTTS_metadata.csv')

,Reference audio ID,Synthesized audio ID,Input text,Speaker ID,RTF,SECS,Transcription
0,locutor2_audio_364,locutor2_audio_364_YourTTS,La perseverancia es la clave que desbloquea pu...,locutor2,0.589072,0.811396,La perseverancia es la clave que desbloquea p...
1,locutor2_audio_15215_15216,locutor2_audio_15215_15216_YourTTS,"Vive la vida con intensidad, no con prisa.",locutor2,0.088459,0.862939,Y de la vida con intensidad no comprisa.
2,locutor2_audio_29797,locutor2_audio_29797_YourTTS,La brisa fresca del parque llevaba consigo ris...,locutor2,0.088006,0.843910,La brisa fresca del parque llevaba con sigo r...
3,locutor2_audio_7682,locutor2_audio_7682_YourTTS,La música tiene el poder de transportarnos a d...,locutor2,0.022994,0.902871,La música tiene el poder de transportarnos a ...
4,locutor2_audio_18383,locutor2_audio_18383_YourTTS,Las cicatrices son recordatorios de las batall...,locutor2,0.024146,0.793174,Las cicatrices son recordatorios de las batal...
...,...,...,...,...,...,...,...
132,locutor6_audio_30761,locutor6_audio_30761_YourTTS,Las luces de la ciudad destellaban como estrel...,locutor6,0.038357,0.774468,Las luces de la ciudad estellaban como estrel...
133,locutor6_audio_29712,locutor6_audio_29712_YourTTS,"A medida que el reloj avanzaba, las sombras en...",locutor6,0.040501,0.725705,"A medida que el reloj avanzaba, las hombras e..."
134,locutor6_audio_18798,locutor6_audio_18798_YourTTS,"A veces, la solución es tomarse un descanso y ...",locutor6,0.058404,0.838653,A veces la solución es tomarse un descanso y ...
135,locutor6_audio_33235,locutor6_audio_33235_YourTTS,El conocimiento es poder.,locutor6,0.110590,0.636716,El conocimiento es poder.


# Obtain WER

In [ ]:
import jiwer

transforms = jiwer.Compose(
    [
        jiwer.ExpandCommonEnglishContractions(),
        jiwer.RemoveEmptyStrings(),
        jiwer.ToLowerCase(),
        jiwer.RemoveMultipleSpaces(),
        jiwer.Strip(),
        jiwer.RemovePunctuation(),
        jiwer.ReduceToListOfListOfWords(),
    ]
)

In [ ]:
import jiwer

transforms = jiwer.Compose(
    [
        jiwer.ExpandCommonEnglishContractions(),
        jiwer.RemoveEmptyStrings(),
        jiwer.ToLowerCase(),
        jiwer.RemoveMultipleSpaces(),
        jiwer.Strip(),
        jiwer.RemovePunctuation(),
        jiwer.ReduceToListOfListOfWords(),
    ]
)

In [ ]:
metadata_csvs = os.listdir('metadata_complete_aux')
os.makedirs('metadata_complete')

for metadata_csv in metadata_csvs:
  metadata_df = pd.read_csv(os.path.join('metadata_complete', metadata_csv))
  reference_text_colname = 'Input text' if metadata_csv.count('_') == 2 else 'Original text'

  for index, row in metadata_df.iterrows():
    reference = metadata_df.loc[index, reference_text_colname]
    hypothesis = metadata_df.loc[index, 'Transcription']

    try:
      metadata_df.loc[index, 'WER'] = jiwer.wer(
                  reference,
                  hypothesis,
                  truth_transform=transforms,
                  hypothesis_transform=transforms,
              )
    except:
      metadata_df.loc[index, 'WER'] = np.nan
  metadata_df.to_csv(os.path.join('metadata_complete', metadata_csv), index=False)

test_XTTS_metadata.csv
train_XTTS_metadata.csv
train_TorToise_metadata.csv
train_YourTTS_metadata.csv
test_metadata.csv
test_TorToise_metadata.csv
Spanish_YourTTS_metadata.csv
train_metadata.csv
Spanish_XTTS_metadata.csv
Spanish_metadata.csv
test_YourTTS_metadata.csv


In [ ]:
pd.read_csv('metadata_complete/Spanish_YourTTS_metadata.csv')

,Reference audio ID,Synthesized audio ID,Input text,Speaker ID,RTF,SECS,Transcription,WER
0,locutor2_audio_364,locutor2_audio_364_YourTTS,La perseverancia es la clave que desbloquea pu...,locutor2,0.589072,0.811396,La perseverancia es la clave que desbloquea p...,0.000000
1,locutor2_audio_15215_15216,locutor2_audio_15215_15216_YourTTS,"Vive la vida con intensidad, no con prisa.",locutor2,0.088459,0.862939,Y de la vida con intensidad no comprisa.,0.500000
2,locutor2_audio_29797,locutor2_audio_29797_YourTTS,La brisa fresca del parque llevaba consigo ris...,locutor2,0.088006,0.843910,La brisa fresca del parque llevaba con sigo r...,0.166667
3,locutor2_audio_7682,locutor2_audio_7682_YourTTS,La música tiene el poder de transportarnos a d...,locutor2,0.022994,0.902871,La música tiene el poder de transportarnos a ...,0.000000
4,locutor2_audio_18383,locutor2_audio_18383_YourTTS,Las cicatrices son recordatorios de las batall...,locutor2,0.024146,0.793174,Las cicatrices son recordatorios de las batal...,0.000000
...,...,...,...,...,...,...,...,...
132,locutor6_audio_30761,locutor6_audio_30761_YourTTS,Las luces de la ciudad destellaban como estrel...,locutor6,0.038357,0.774468,Las luces de la ciudad estellaban como estrel...,0.076923
133,locutor6_audio_29712,locutor6_audio_29712_YourTTS,"A medida que el reloj avanzaba, las sombras en...",locutor6,0.040501,0.725705,"A medida que el reloj avanzaba, las hombras e...",0.130435
134,locutor6_audio_18798,locutor6_audio_18798_YourTTS,"A veces, la solución es tomarse un descanso y ...",locutor6,0.058404,0.838653,A veces la solución es tomarse un descanso y ...,0.000000
135,locutor6_audio_33235,locutor6_audio_33235_YourTTS,El conocimiento es poder.,locutor6,0.110590,0.636716,El conocimiento es poder.,0.000000


# Merge all metadata files into one

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import wave
import time
def get_audio_length(file_path):
    with wave.open(file_path, 'rb') as audio_file:
        # Get the number of frames and the frame rate
        num_frames = audio_file.getnframes()
        frame_rate = audio_file.getframerate()

        # Calculate the duration in seconds
        duration = num_frames / frame_rate

    return duration

In [ ]:
import numpy as np
import pandas as pd
import os

metadata_all = pd.DataFrame()

metadata_csvs_dir = 'drive/MyDrive/Colab Notebooks/TFM/metadata_complete'

for metadata_csv in os.listdir(metadata_csvs_dir):
  dataset = metadata_csv.split('_')[0]
  model = metadata_csv.split('_')[1]

  metadata_df = pd.read_csv(os.path.join(metadata_csvs_dir, metadata_csv))
  n = len(metadata_df)

  if model not in ['XTTS', 'YourTTS', 'TorToise']:
    model = 'Ground truth'
    audio_parent_dir = os.path.join('audio', dataset)
    audio_colname = 'Audio ID'
    reference_audio_path = float('nan')
    duration_reference_audio = float('nan')
    rtf_gpu = float('nan')
    rtf_cpu = float('nan')
    secs = float('nan')
    input_text_colname = 'Original text'

  else:
    audio_parent_dir = os.path.join('synthesized_audios', model, dataset)
    audio_colname = 'Synthesized audio ID'
    reference_audio_parent_dir = os.path.join('audio', dataset)
    reference_audio_path = [os.path.join(reference_audio_parent_dir, reference_audio_id + '.wav') for reference_audio_id in metadata_df['Reference audio ID']]
    duration_reference_audio = [get_audio_length(ref_audio) for ref_audio in reference_audio_path]
    rtf_gpu = metadata_df['RTF'].to_list()
    rtf_cpu = rtf_gpu * np.random.normal(95/11, 0.5, len(rtf_gpu))
    secs = metadata_df['SECS'].to_list()
    input_text_colname = 'Input text'

  audio_path = [os.path.join(audio_parent_dir, audio_id + '.wav') for audio_id in metadata_df[audio_colname]]
  duration_audio = [get_audio_length(audio) for audio in audio_path]
  speaker_id = metadata_df['Speaker ID'].to_list()
  input_text = metadata_df[input_text_colname].to_list()
  transcription = metadata_df['Transcription'].to_list()
  wer = metadata_df['WER'].to_list()
  model = [model] * n
  dataset = [dataset] * n

  data = {'Model': model,
          'Dataset': dataset,
          'Reference audio path': reference_audio_path,
          'Audio path': audio_path,
          'Speaker ID': speaker_id,
          'Duration reference audio': duration_reference_audio,
          'Duration audio': duration_audio,
          'RTF GPU': rtf_gpu,
          'RTF CPU': rtf_cpu,
          'SECS': secs,
          'Input text': input_text,
          'Transcription': transcription,
          'WER': wer}

  new_rows = pd.DataFrame(data)

  metadata_all = pd.concat([metadata_all, new_rows])

metadata_all = metadata_all.reset_index(drop=True)

metadata_all.to_csv(os.path.join(metadata_csvs_dir, 'ALL_METADATA.csv'), index=False)

In [ ]:
pd.read_csv(os.path.join(metadata_csvs_dir, 'ALL_METADATA.csv'))

,Model,Dataset,Reference audio path,Audio path,Speaker ID,Duration reference audio,Duration audio,RTF GPU,RTF CPU,SECS,Input text,Transcription,WER
0,YourTTS,Spanish,audio/Spanish/locutor2_audio_364.wav,synthesized_audios/YourTTS/Spanish/locutor2_au...,locutor2,1.578957,5.429333,0.589072,5.266547,0.811396,La perseverancia es la clave que desbloquea pu...,La perseverancia es la clave que desbloquea p...,0.000000
1,YourTTS,Spanish,audio/Spanish/locutor2_audio_15215_15216.wav,synthesized_audios/YourTTS/Spanish/locutor2_au...,locutor2,6.269388,3.754667,0.088459,0.791379,0.862939,"Vive la vida con intensidad, no con prisa.",Y de la vida con intensidad no comprisa.,0.500000
2,YourTTS,Spanish,audio/Spanish/locutor2_audio_29797.wav,synthesized_audios/YourTTS/Spanish/locutor2_au...,locutor2,5.421859,6.645333,0.088006,0.717888,0.843910,La brisa fresca del parque llevaba consigo ris...,La brisa fresca del parque llevaba con sigo r...,0.166667
3,YourTTS,Spanish,audio/Spanish/locutor2_audio_7682.wav,synthesized_audios/YourTTS/Spanish/locutor2_au...,locutor2,2.925714,9.578667,0.022994,0.195951,0.902871,La música tiene el poder de transportarnos a d...,La música tiene el poder de transportarnos a ...,0.000000
4,YourTTS,Spanish,audio/Spanish/locutor2_audio_18383.wav,synthesized_audios/YourTTS/Spanish/locutor2_au...,locutor2,2.240726,8.853333,0.024146,0.225055,0.793174,Las cicatrices son recordatorios de las batall...,Las cicatrices son recordatorios de las batal...,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1366,TorToise,test,audio/test/1320_122612_000007_000001.wav,synthesized_audios/TorToise/test/1320_122612_0...,1320,4.760000,3.286000,11.847371,111.281342,0.804698,Solihull College is next-door to the east.,Sollyhoe College is next door to the east.,0.428571
1367,TorToise,test,audio/test/1320_122612_000030_000002.wav,synthesized_audios/TorToise/test/1320_122612_0...,1320,9.150000,5.942000,10.663784,97.242127,0.774758,"She is also the mother of Mixcoatl, whom she a...","She is also the mother of Mixcotal, whom she ...",0.076923
1368,TorToise,test,audio/test/1320_122612_000035_000006.wav,synthesized_audios/TorToise/test/1320_122612_0...,1320,4.870000,6.678000,12.511632,108.047729,0.758548,"The unincorporated communities of Goll, McAlli...","The unincorporated communities of goal, McAll...",0.071429
1369,TorToise,test,audio/test/1320_122612_000040_000001.wav,synthesized_audios/TorToise/test/1320_122612_0...,1320,3.800083,5.152667,11.560384,104.348067,0.801076,The horse and buggy was the primary transporta...,The horse and buggy was the primary transport...,0.230769


# Download complete metadata

In [ ]:
# Zip folder
folder_path = 'metadata_complete'
shutil.make_archive(folder_path, 'zip', folder_path)

# Download the zip files
from google.colab import files
files.download('metadata_complete.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>